# Testing CAPPI data with Py-ART and wradlib

@author: Camila Lopes (camila.lopes@iag.usp.br)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4
import xarray as xr
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.io.shapereader import Reader

import pyart
import wradlib as wrl

from read_sipam_cappis import read_sipam_cappi

In [ ]:
def ppi_to_grid(filename):

    # Read data and convert to grid
    radar = pyart.aux_io.read_gamic(filename)
    grid = pyart.map.grid_from_radars(
        radar, grid_shape=(20, 501, 501),
        grid_limits=((1e3, 20e3), (-250e3, 250e3), (-250e3, 250e3))
    )

    return grid

In [ ]:
filename = "../data/radar/sipam_manaus/arm_cappi/2014-01/20140103/sbmn_cappi_20140103_160011.nc"
wset = wrl.io.read_generic_netcdf(filename)

In [ ]:
plt.imshow(wset['variables']['DBZc']['data'][0, 20, :, :])

In [ ]:
xset = xr.open_dataset(filename)
xset.variables['z0']

In [ ]:
dset = netCDF4.Dataset(filename)
dset

In [ ]:
dset.variables['time_bounds']

In [ ]:
grid = ppi_to_grid("../data/radar/sipam_manaus/arm/201401/RADL08061720140103031200.HDF5")

In [ ]:
cappi = read_sipam_cappi(filename)

In [ ]:
str(cappi.z['data']/1000)

In [ ]:
pyart.util.datetime_from_grid(cappi).strftime('%Y%m%d%H%M%S')

In [ ]:
np.array(np.ma.MaskedArray.tolist(cappi.fields['DBZc']['data']))
# np.nanmax(x[x != None])

In [ ]:
display = pyart.graph.GridMapDisplay(cappi)
projection = ccrs.PlateCarree()
display.plot_grid('DBZc', level=0, projection=projection)

In [ ]:
display.grid.to_xarray()

In [ ]:
raw = pyart.aux_io.read_gamic(
    "../data/radar/sipam_manaus/arm/201401/RADL08061720140103031200.HDF5")

"%.2f" % raw.fixed_angle['data'][0]

In [ ]:
ranges = raw.range['data']
elevs = raw.fixed_angle['data']
site = (float(raw.longitude['data']),
        float(raw.latitude['data']),
        float(raw.altitude['data']))
beamwidth = float(raw.instrument_parameters['radar_beam_width_h']['data'])


ax = wrl.vis.plot_scan_strategy(ranges, elevs, site, beamwidth, vert_res=1000, maxalt=20000, units='km')
ax.set_title('SIPAM S-Band')

In [ ]:
# GoAmazon site locations
sites = pd.read_csv("../data/general/goamazon_sites.csv", sep=";", decimal=",")
sites['Latitude']

In [ ]:
# Range rings + site locations
display = pyart.graph.RadarMapDisplay(raw)
fig = plt.figure(figsize = (6,6))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())

display.plot_ppi_map("corrected_reflectivity", 0, vmin=0, shapefile="../data/general/shapefiles/lineaire_1km",
    shapefile_kwargs={"facecolor": "None", "edgecolor": "darkblue", "alpha": 0.5, "linewidth": 0.75})
# GoAmazon sites
ax.scatter(sites["Longitude"], sites["Latitude"], s=5, c='red')
ax.annotate(sites["GoAmazon2014/5 reference"][0], xy=(sites["Longitude"][0], sites["Latitude"][0]))
ax.annotate(sites["GoAmazon2014/5 reference"][1], xy=(sites["Longitude"][1], sites["Latitude"][1]))
ax.annotate(sites["GoAmazon2014/5 reference"][2], xy=(sites["Longitude"][2], sites["Latitude"][2]))
ax.annotate(sites["GoAmazon2014/5 reference"][3], xy=(sites["Longitude"][3], sites["Latitude"][3]), horizontalalignment='right')
ax.annotate(sites["GoAmazon2014/5 reference"][4], xy=(sites["Longitude"][4], sites["Latitude"][4]), horizontalalignment='right')
ax.annotate(sites["GoAmazon2014/5 reference"][5], xy=(sites["Longitude"][5], sites["Latitude"][5]))
ax.annotate(sites["GoAmazon2014/5 reference"][6], xy=(sites["Longitude"][6], sites["Latitude"][6]), horizontalalignment='right')
ax.annotate(sites["GoAmazon2014/5 reference"][7], xy=(sites["Longitude"][7], sites["Latitude"][7]))
ax.annotate(sites["GoAmazon2014/5 reference"][8], xy=(sites["Longitude"][8], sites["Latitude"][8]), horizontalalignment='right')
# Adding shapefile
ax.add_geometries(Reader("../data/general/shapefiles/AM_Municipios_2019").geometries(), ccrs.PlateCarree(), linewidth=0.75,
    facecolor="None", edgecolor="darkgray", alpha=0.8)
# Adding gridlines
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, xlocs=np.arange(-70, -50, 1), ylocs=np.arange(-10, 1, 1), alpha=0.5)
gl.top_labels = gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
display.plot_range_rings([50, 100, 150, 200, 250], ax=ax, col="black", lw=1)
plt.savefig("figs/sipam_range_rings.png", dpi=300, bbox_inches="tight")
